In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-238954
Azure region: eastus2
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-238954


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "Udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing cluster")
except ComputeTargetException:
    print("Creating new cluster")

    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'STANDARD_D2_V2',
        min_modes=0,
        max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Found existing cluster

Running


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': choice(0.1,0.5,1,5),
        '--max_iter': choice(50, 100, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./',
                script='train_wv.py',
                compute_target='Udacity',
                environment=sklearn_env)

#primary_metric = 'Accuracy'
# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                hyperparameter_sampling=ps,
                policy=policy,
                primary_metric_name='Accuracy',
                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                max_total_runs=100,
                max_concurrent_runs=4)

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c
Web View: https://ml.azure.com/runs/HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-238954/workspaces/quick-starts-ws-238954&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-19T16:03:32.654181][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-19T16:03:33.1101013Z][SCHEDULER][INFO]Scheduling job, id='HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_0' 
[2023-07-19T16:03:33.2698136Z][SCHEDULER][INFO]Scheduling job, id='HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_1' 
[2023-07-19T16:03:33.3096906Z][SCHEDULER][INFO]Scheduling job, id='HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_2' 
[2023-07-19T16:03:33.4021500Z][SCHEDULER][INFO]Scheduling job, id='HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_3' 
[2023-07-19T16:03:33.346560][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c',
 'target': 'Udacity',
 'status': 'Completed',
 'startTimeUtc': '2023-07-19T16:03:31.848877Z',
 'endTimeUtc': '2023-07-19T16:08:36.114441Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1640d5fd-26e8-4189-b7b2-01978ea8a402',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '12',
  'score': '0.9118718135469774',
  'best_child_run_id': 'HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType'

In [8]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run)
best_run.register_model(model_name = 'best_hd_model.pkl', model_path='outputs/model.joblib')

Run(Experiment: udacity-project,
Id: HD_8a0dde87-aa8e-46d8-be4d-1f17f48ff36c_1,
Type: azureml.scriptrun,
Status: Completed)


Model(workspace=Workspace.create(name='quick-starts-ws-238954', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-238954'), name=best_hd_model.pkl, id=best_hd_model.pkl:1, version=1, tags={}, properties={})

In [9]:
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '5', '--max_iter', '100']


In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [11]:

from train_wv import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(data)

data_joined = pd.concat([x, y], axis=1)

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=data_joined,
    label_column_name='y',
    n_cross_validations=2)

In [16]:
# Submit your automl run

experiment = Experiment(ws, 'automl')
run = experiment.submit(config=automl_config, show_out=True)

2023-07-19:16:14:29,978 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_470ea814-f373-47c7-9a35-7c2e44eec849,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/07/19 16:21:40 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-07-19:16:46:59,8 INFO     [explanation_client.py:334] Using default datastore for uploads


In [17]:
# Retrieve and save your best automl model.

best_run, model = run.get_output()
best_run.register_model(model_name = 'automl_best_run', model_path='outputs/')
print(best_run)

Run(Experiment: automl,
Id: AutoML_470ea814-f373-47c7-9a35-7c2e44eec849_30,
Type: None,
Status: Completed)
